In [846]:
import math
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from pulp import *

In [847]:
img1 = Image.open("test1.jpg")
img2 = Image.open("test2.jpg")

In [848]:
img1_array = np.array(img1)
img2_array = np.array(img2)

In [849]:
print(img1_array.shape)
print(img1_array.size)
print(img1_array[0][0])
print(img1_array[0][0][0])

(213, 320, 3)
204480
[104 105 100]
104


In [850]:
pixel_matrix = img1_array.reshape(-1, 3)
pixel_matrix2 = img2_array.reshape(-1, 3)
print(pixel_matrix2)

[[194 232 211]
 [198 236 215]
 [197 232 210]
 ...
 [ 93  98  66]
 [ 75  85  51]
 [ 64  73  46]]


In [851]:
number_cluster = 3
kmeans1 = KMeans(n_clusters=number_cluster, random_state=0).fit(pixel_matrix)
kmeans2 = KMeans(n_clusters=number_cluster, random_state=0).fit(pixel_matrix2)

In [852]:
new_img_array = np.zeros_like(pixel_matrix)
for i, label in enumerate(kmeans1.labels_):
    new_img_array[i] = kmeans1.cluster_centers_[label]
new_img = new_img_array.reshape(img1_array.shape)

new_img_array2 = np.zeros_like(pixel_matrix2)
for i, label in enumerate(kmeans2.labels_):
    new_img_array2[i] = kmeans2.cluster_centers_[label]
new_img2 = new_img_array2.reshape(img2_array.shape)

In [853]:
Image.fromarray(new_img).show()
Image.fromarray(new_img).save('clustered1_' + str(number_cluster) + 'clust.jpg')

In [854]:
Image.fromarray(new_img2).show()
Image.fromarray(new_img2).save('clustered2_' + str(number_cluster) + 'clust.jpg')

In [855]:
""" 1ère étape : Définir notre matrice de coût avec les distances euclidiennes entre les clusters_centers_ 
    2ème étape : Définir les masses de chacun des clusters en utilisant np.count_nonzero(kmeans1.labels_ == i) pour i allant de 0 à kmeans1.cluster_centers_.shape[0]
    qu'on divisera, si nécessaire, par le nombre total de pixel de l'image (qu'on obtient par exemple avec img_array.size // 3) pour construire les arrays des masses des deux images
    3ème étape : On résout le problème de transport optimal avec Pulp
    4ème étape : Avec le plan de transport/matrice de couplage, pour l'instant, on implémente une fonction qui à partir de la deuxième image effectue une permutation de ses pixels.
    Plus précisément, on itére sur les tous chemins/routes possibles, et on pour chaque chemin, i.e pour le cluster i de la première image vers le cluster j de la deuxième,
    on regarde le nombre de pixels à transporter et on les transporte.
"""

" 1ère étape : Définir notre matrice de coût avec les distances euclidiennes entre les clusters_centers_ \n    2ème étape : Définir les masses de chacun des clusters en utilisant np.count_nonzero(kmeans1.labels_ == i) pour i allant de 0 à kmeans1.cluster_centers_.shape[0]\n    qu'on divisera, si nécessaire, par le nombre total de pixel de l'image (qu'on obtient par exemple avec img_array.size // 3) pour construire les arrays des masses des deux images\n    3ème étape : On résout le problème de transport optimal avec Pulp\n    4ème étape : Avec le plan de transport/matrice de couplage, pour l'instant, on implémente une fonction qui à partir de la deuxième image effectue une permutation de ses pixels.\n    Plus précisément, on itére sur les tous chemins/routes possibles, et on pour chaque chemin, i.e pour le cluster i de la première image vers le cluster j de la deuxième,\n    on regarde le nombre de pixels à transporter et on les transporte.\n"

In [856]:
def get_cost_matrix(cluster_start, cluster_arrival):

    height, width = cluster_start.shape[0], cluster_arrival.shape[0]
    
    res = np.zeros((height, width), dtype=np.float16)

    for i in range(height):
        for j in range(width):
            res[i, j] = math.dist(cluster_start[i], cluster_arrival[j])
    return res


In [857]:
test = np.zeros((2, 3), dtype=np.float16)

In [858]:
cost_matrix = get_cost_matrix(kmeans1.cluster_centers_, kmeans2.cluster_centers_)
print(cost_matrix, cost_matrix.shape)

[[104.7   33.8  229.  ]
 [ 33.22 144.8  157.9 ]
 [118.25 180.5   81.25]] (3, 3)


In [859]:
m_start = [np.count_nonzero(kmeans1.labels_ == i) for i in range(kmeans1.cluster_centers_.shape[0])]
m_arrival = [np.count_nonzero(kmeans2.labels_ == i) for i in range(kmeans2.cluster_centers_.shape[0])]
m = m_start.copy()
m[0] = 3
print(m, m_start)

[3, 23416, 19344] [25400, 23416, 19344]


In [860]:
def optimal_solve(costs, m_start, m_arrival):
  # Initialise les variables du probleme
  problem = LpProblem("Optimal_transport", LpMinimize)
  paths = [(i, j) for i in range(len(m_start)) for j in range(len(m_arrival))]
  path_variables = LpVariable.dicts("Path", (range(len(m_start)), range(len(m_arrival))),
                                lowBound=0, cat='Integer')

  # Definit la fonction objective
  problem += lpSum([path_variables[i][j] * costs[i][j] for (i, j) in paths])

  # Pose les contraintes
  for i in range(len(m_start)):
      problem += lpSum([path_variables[i][j] for j in range(len(m_arrival))]) == m_start[i]
  for j in range(len(m_arrival)):
      problem += lpSum([path_variables[i][j] for i in range(len(m_start))]) == m_arrival[j]
  # Utilise la méthode solve() associee à l'objet de type LpProblem pour resoudre le problème de programmation lineaire  
  problem.solve()

  # Stocke les resultats dans un dictionnaire
  result = {}
  for v in problem.variables():
      route = v.name.split("_")[1:]
      i, j = int(route[0]), int(route[1])
      result[i, j] = v.varValue
  # Construit la matrice de couplage / le plan de transport optimal  
  optimal_plan = np.zeros((len(m_start), len(m_arrival)))
  for i, j in result.keys():
    optimal_plan[i,j] = result[i, j]
  return optimal_plan


In [861]:
optimal_plan = optimal_solve(cost_matrix, m_start, m_arrival)
print(optimal_plan)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Dec  8 2020 

command line - cbc /tmp/41d45748ed6640068b82e32df6cd887b-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/41d45748ed6640068b82e32df6cd887b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 57 RHS
At line 64 BOUNDS
At line 74 ENDATA
Problem MODEL has 6 rows, 9 columns and 18 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3.52347e+06 - 0.00 seconds
Cgl0004I processed model has 6 rows, 9 columns (9 integer (0 of which binary)) and 18 elements
Cutoff increment increased from 1e-05 to 0.0312187
Cbc0012I Integer solution of 3523471.2 found by greedy equality after 0 iterations and 0 nodes (0.01 seconds)
Cbc0001I Search completed - best objective 3523471.25, took 0 iterations and 0 nodes (0.01 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root no

In [862]:
def map_labels(img_array):
    map = {}
    size = img_array.size // 3
    


In [863]:
def recolorized_image_array(img1_array, pixel_matrix2, optimal_plan, kmeans1, kmeans2, m_start):
        
    res = np.zeros_like(pixel_matrix2)
    unavailable_mass = [0 for l in range(len(m_start))]
    for j in range(kmeans2.cluster_centers_.shape[0]):
        counter = 0
        arg_cluster = np.where(kmeans2.labels_ == j)[0]
        for i in range(kmeans1.cluster_centers_.shape[0]):
            if optimal_plan[i][j] != 0.:
                target_positions = np.where(kmeans1.labels_ == i)[0][unavailable_mass[i]: unavailable_mass[i] + int(optimal_plan[i][j])]
                print(target_positions)
                for pos in target_positions:
                    #print(pixel_matrix2[arg_cluster[counter]])
                    #print(res.shape, pixel_matrix2.shape)
                    #print(pos)
                    #print(pos)
                    #print(counter)
                    #print(arg_cluster[counter])
                    #print(res[pos], pixel_matrix2[arg_cluster[counter]])
                    res[pos] = pixel_matrix2[arg_cluster[counter]]
                    counter += 1
                unavailable_mass[i] += int(optimal_plan[i][j])
    new_img = res.reshape(img1_array.shape)
    return new_img

In [864]:
test_img = recolorized_image_array(img1_array, pixel_matrix2, optimal_plan, kmeans1, kmeans2, m_start)

[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22  124  125  126  127  128
  129  130  131  132  133  149  150  151  152  153  154  155  156  157
  158  159  160  161  162  163  164  165  166  167  168  169  170  171
  172  173  174  175  176  177  178  179  180  181  182  183  184  185
  186  187  188  189  190  194  195  196  234  235  236  237  238  239
  240  241  242  243  244  320  321  322  323  324  325  326  327  328
  329  330  331  332  333  334  335  336  337  338  339  340  341  342
  343  428  429  430  431  432  433  434  435  436  437  438  439  440
  441  442  443  444  445  446  447  448  449  472  473  474  475  476
  477  478  479  480  481  482  483  484  485  486  487  488  489  490
  491  503  504  505  506  555  556  557  558  559  560  561  562  563
  564  565  591  593  594  595  596  597  598  599  603  604  605  606
  607  608  609  610  611  612  613  614  640  641  642  643  644  645
  646 

In [865]:
Image.fromarray(test_img).show()
Image.fromarray(test_img).save('planete7.jpg')